## Agents

### 🤖 What is an Agent?

- An LLM configured with specific instructions to do a task. It may use tools to achieve more specific goals.

#### install

In [ ]:
# ! pip install openai-agents nest_asyncio


### imports

In [25]:
from pydantic import BaseModel
import dotenv
import nest_asyncio
from agents import Agent, Runner, GuardrailFunctionOutput, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import asyncio
from typing import AsyncGenerator
import requests
import json
from IPython.display import display, Markdown, DisplayHandle
import os

##### Apply nest_asyncio to allow asyncio event loops to be nested

In [3]:
nest_asyncio.apply()

#### Load environment variables

In [26]:
dotenv.load_dotenv()

GPT_4_1_NANO = os.getenv('CHEAP_AND_BEST_MODEL', 'gpt-4.1-nano')

##### function tool

In [5]:
@function_tool
def get_usd_base_rate() -> dict:
  """
  Get today's USD base rate.
  """
  response: dict = requests.get("https://api.vatcomply.com/rates?base=USD")
  if response.status_code != 200:
    return f"Error: {response.status_code}"
  return response.json()

### Agent with tool

In [ ]:
currency_agent = Agent(
  name="Currency Agent",
  model=GPT_4_1_NANO,
  tools=[get_usd_base_rate],
  instructions="""You are an expert financial assistant, with expertise in currency conversion. To get the latest USD base rate, use the get_usd_base_rate function.
Provide the user with answers in their requested currency and USD equivalent.

Example: 
User: How much does it cost to buy a tesla in London?
Assistant: The cost of a Tesla in London is £50,000. This is equivalent to $65,000 USD based on the current exchange rate.
"""
)

### Ask the agent

In [14]:
async def ask_agent(q: str, agent: Agent) -> AsyncGenerator[str, None]:      
    result = Runner.run_streamed(agent, input=q)
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            yield event.data.delta

##### question

In [8]:
question: str = "How much does it cost to buy a tesla in Canada?"

#### Streaming Response (with Markdown)

In [16]:
async def stream_response(q: str, agent: Agent) -> AsyncGenerator[str, None]:
    """
    Stream the response from the agent.
    """
    full_response = ""
    display_handle = display(Markdown(full_response), display_id=True)
    async for chunk in ask_agent(q, agent):
      full_response += chunk
      display_handle.update(Markdown(full_response))
      await asyncio.sleep(0.01)

In [21]:
asyncio.run(stream_response(question, currency_agent))

The cost of a Tesla in Canada is approximately CAD 70,000. Based on the current exchange rate, this is equivalent to about $50,500 USD. 

(Exchange rate: 1 CAD = 0.722 USD)

### Handoffs

##### What is a handoff 🔄
- A mechanism that allows agents to transfer tasks to one or more other agents.
- LLMs perceive handoffs just like any other tool.

##### NOTE: 
- A common pattern includes an initial triage/routing agent followed by specialized agents, with optional escalation to a human or more advanced agent.

##### Example: Handoffs in Travel Agents

A handoff allows an agent to transfer a user's request to another specialized agent. In the travel domain, this could mean routing a query to a flight booking agent, hotel booking agent, or itinerary planner.

In [28]:
from agents import Agent, handoff

flight_agent = Agent(
  name="Flight Booking Agent", 
  instructions="Book flights and answer flight-related queries.",
  model=GPT_4_1_NANO
  )
hotel_agent = Agent(
  name="Hotel Booking Agent", instructions="Book hotels and answer hotel-related queries.",
  model=GPT_4_1_NANO
)
itinerary_agent = Agent(
  name="Itinerary Planner Agent", instructions="Plan travel itineraries and suggest activities.",
  model=GPT_4_1_NANO
)

travel_triage_agent = Agent(
    name="Travel Triage Agent",
    instructions="Route the user's request to the appropriate travel agent: flight, hotel, or itinerary.",
    handoffs=[
        handoff(flight_agent),
        handoff(hotel_agent),
        handoff(itinerary_agent)
    ],
    model=GPT_4_1_NANO
)

### Example: Routing a User Query
Let's see how a user's request is routed to the right agent.

In [ ]:
async def ask_travel_ai(q: str):
    result = await Runner.run(travel_triage_agent, q)
    print(result.final_output)

# Example queries
await ask_travel_ai("Book me a flight from New York to Tokyo next week.")
await ask_travel_ai("Find a hotel in Paris for 3 nights.")
await ask_travel_ai("Plan a 4-day itinerary in Rome.")

### Example: Streaming Response
Let's see how a triage agent can stream responses while routing travel-related queries to the appropriate specialized agent.

In [ ]:
asyncio.run(stream_response(agent=travel_triage_agent, q="Book me a flight from New York to Tokyo next week."))
asyncio.run(stream_response(agent=travel_triage_agent, q="Find a hotel in Paris for 3 nights."))
asyncio.run(stream_response(agent=travel_triage_agent, q="Plan a 4-day itinerary in Rome."))

I will now connect you to a flight booking agent to assist with your flight reservation from New York to Tokyo next week.

I can help you find a hotel in Paris for 3 nights. Do you have specific preferences or requirements, such as the dates of stay, budget, or preferred location within Paris?

Certainly! Here's a suggested 4-day itinerary for Rome, balancing historical sights, cultural experiences, and leisure:

### Day 1: Ancient Rome & the Colosseum
- **Morning:**
  - Visit the **Colosseum**; consider booking a guided tour to explore its history deeply.
  - Walk through the **Roman Forum** and **Palatine Hill** nearby to see ancient ruins.
- **Afternoon:**
  - Have lunch in the Monti district, known for its cozy cafes.
  - Explore the **Pantheon**, one of Rome's best-preserved ancient buildings.
- **Evening:**
  - Stroll around Piazza Navona and enjoy dinner at a local trattoria.

### Day 2: Vatican City & Surroundings
- **Morning:**
  - Tour the **Vatican Museums**, including the **Sistine Chapel**.
  - Visit **St. Peter's Basilica** and if interested, climb to the dome for panoramic views.
- **Afternoon:**
  - Walk across the **Vatican Gardens** or relax in **St. Peter’s Square**.
  - Enjoy lunch near the Vatican.
- **Evening:**
  - Explore the Trastevere neighborhood, famous for lively streets and authentic Roman cuisine.

### Day 3: Artistic & Baroque Rome
- **Morning:**
  - Visit **Piazza del Popolo** and **Spanish Steps**.
  - Explore **Villa Borghese Park** and visit the **Galleria Borghese** (reserve tickets in advance).
- **Afternoon:**
  - Walk through via del Corso for shopping.
  - Discover **Trevi Fountain** and toss a coin for good luck.
- **Evening:**
  - Enjoy dinner in the Campo de' Fiori area with lively nightlife.

### Day 4: Off-the-Beaten-Path & Local Flavors
- **Morning:**
  - Visit **Testaccio** neighborhood for authentic food markets like **Mercato di Testaccio**.
  - Explore the **Non-Catholic Cemetery** and **Cestius Pyramid**.
- **Afternoon:**
  - Take a leisurely walk along the **Appian Way (Via Appia Antica)**, exploring ancient ruins and aqueducts.
  - Optional: rent a bike or walk down this historic route.
- **Evening:**
  - End your trip with a farewell dinner in Trastevere or another charming bairro.

Would you like recommendations for specific restaurants, tours, or accommodations?